In [2]:
%cd ".."
import pyLDAvis
import json
import pandas as pd

c:\Users\atero\Documents\Git\projet-statistique-2022


C:\Users\atero\AppData\Local\Programs\Python\Python310\lib\site-packages\gensim\matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


Chargement des documents

In [3]:
with open('data/docs.json', encoding = "utf8") as f:
    docs: list[str] = json.load(f)
print(f"Nombre de documents : {len(docs)}")

Nombre de documents : 9501


In [4]:
with open('data/liste_lemmes.txt') as f:
    vocabulaire = f.readlines()

# Suppression des retours chariots sur chaque ligne
for i in range(len(vocabulaire)):
    vocabulaire[i] = vocabulaire[i].replace('\n', '')

print(f"Taille du vocabulaire initial : {len(vocabulaire)}")

Taille du vocabulaire initial : 7231


Calcul du nombre d'apparition de chaque mot, on conserve seulement ceux présents un certain nombre de fois

In [5]:
# Nombre de fois où chaque mot apparaît
term_frequency_corpus: dict = {mot: 0 for mot in vocabulaire}
for doc in docs:
    for mot in doc:
        term_frequency_corpus[mot] += 1

# Restriction du vocabulaire et des documents en supprimant certains mots
term_freq_min: int = 10
vocabulaire = [mot for mot in vocabulaire if term_frequency_corpus[mot] >= term_freq_min]
for i in range(len(docs)):
   docs[i] = [mot for mot in docs[i] if mot in vocabulaire]

term_frequency_corpus = {mot: term_frequency_corpus[mot] for mot in vocabulaire}
term_frequency_corpus_standard: dict = {mot: term_frequency_corpus[mot] / sum(term_frequency_corpus.values()) for mot in term_frequency_corpus}
print(f"Nombre total de mots : {sum(term_frequency_corpus.values())}")

Nombre total de mots : 57655


Chargement des valeurs génériques à toutes les représentations de modèles

In [6]:
args_pyLDA = {'vocab': vocabulaire, # liste du vocabulaire,
              'R': 10, # Taille du "TOP" à afficher
              'term_frequency': list(term_frequency_corpus.values()), # Fréquence absolue de chaque mot
              'n_jobs': -1, # Parallélisme
              'lambda_step': 1 # Calcul pour lambda = 0 et lambda = 1
            }

Chargement des labels de chaque document pour chaque clustering


In [7]:
with open("data/tuning/clustering/best_labels.json") as f:
    labelsTousModeles: dict = json.load(f)['ReductionFirst']

In [83]:
def preparation_LDAvis(clustersDocs: list[int]) -> pyLDAvis.PreparedData:
    
    indicesDocsClasses = [i for i in range(len(docs)) if clustersDocs[i] != -1]
    docsClasses = [docs[i] for i in indicesDocsClasses]
    clustersDocsClasses = [clustersDocs[i] for i in indicesDocsClasses]
    nomsClusters = set(clustersDocsClasses)
    K: int = len(nomsClusters)
    
    
    # Probabilité que chaque mot appartienne à un cluster / topic
    term_frequency_topics: pd.DataFrame = pd.DataFrame(columns = vocabulaire, index = nomsClusters, dtype = float)
    term_frequency_topics.loc[:,:] = 0

    for i, doc in enumerate(docsClasses):
        for mot in doc:
            if mot in vocabulaire:
                term_frequency_topics.loc[clustersDocsClasses[i], mot] += 1. / term_frequency_corpus[mot]
    
    # Probabilité que chaque mot appartienne à un cluster / topic
    term_frequency_topics: pd.DataFrame = pd.DataFrame(columns = vocabulaire, index = range(K), dtype = float)
    term_frequency_topics.loc[:,:] = 0

    for i, doc in enumerate(docsClasses):
        for mot in doc:
            if mot in vocabulaire:
                term_frequency_topics.loc[clustersDocsClasses[i], mot] += 1. / term_frequency_corpus[mot]
    
    # Probabilité qu'un document appartienne à un cluster / topic (soit 0 soit 1 pour notre cas)
    topic_docs_matrice: pd.DataFrame = pd.DataFrame(columns = range(K), index = range(len(indicesDocsClasses)), dtype = float)
    topic_docs_matrice.loc[:,:] = 0

    for i in range(len(indicesDocsClasses)):
        topic_docs_matrice.loc[i,clustersDocsClasses[i]] = 1.
                
                
    visualisation = pyLDAvis.prepare(topic_term_dists = term_frequency_topics.to_numpy(),
                                 doc_topic_dists = topic_docs_matrice.to_numpy(),
                                 doc_lengths = [len(doc) for doc in docsClasses], **args_pyLDA)
    return visualisation
    

# Modèle CBOW

Chargement de modèle et de labels clusters. On ne conserve que les documents qui sont dans catégorisés

In [84]:
clustersDocs: list[int] = labelsTousModeles['cbow']['silhouette']
visualisation = preparation_LDAvis(clustersDocs)

C:\Users\atero\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
C:\Users\atero\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
C:\Users\atero\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [85]:
pyLDAvis.save_html(visualisation, "data/LDAvis/CBOW.html")
pyLDAvis.save_json(visualisation, "data/LDAvis/CBOW.json")
pyLDAvis.display(visualisation)

# Modèle Skip-gram

Chargement de modèle et de labels clusters. On ne conserve que les documents qui sont dans catégorisés

In [100]:
clustersDocs: list[int] = labelsTousModeles['skipgram']['silhouette']
visualisation = preparation_LDAvis(clustersDocs)

C:\Users\atero\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
C:\Users\atero\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
C:\Users\atero\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [15]:
pyLDAvis.save_html(visualisation, "data/LDAvis/Skip_Gram.html")
pyLDAvis.save_json(visualisation, "data/LDAvis/Skip_Gram.json")
pyLDAvis.display(visualisation)

In [122]:
for i, doc in enumerate(docs):
    if 'abandon' in doc:
        print('X' if clustersDocs[i] == 0 else '', end = '')
        print(doc, end = '')
        print('[' + str(clustersDocs[i]) + ']')

['obliger', 'maire', 'reboiser', 'systematiquement', 'terrain', 'abandon', 'cesser'][3]
['vegetaliser', 'espace', 'urbain', 'public', 'parking', 'abandon', 'developper', 'espace', 'verts', 'ville'][1]
['animal', 'domestique', 'produit', 'animal', 'nourriture', 'abandon'][2]
['sensibiliser', 'proprietaire', 'vendre', 'bien', 'laisser', 'vide', 'abandon', 'beton'][6]
X['mettre', 'place', 'obligatoire', 'animal', 'moindre', 'eviter', 'abandon'][0]
X['mettre', 'place', 'permis', 'animal', 'contre', 'abandon'][0]
['punir', 'severement', 'abandon', 'animal'][1]
X['mettre', 'place', 'action', 'population', 'abandon', 'dechet', 'nature'][0]
X['mettre', 'place', 'permis', 'animal', 'lutter', 'contre', 'abandon'][0]
X['mettre', 'place', 'kilomètre', 'velo', 'encourager', 'abandon', 'voiture'][0]


# Modèle GloVe

Chargement de modèle et de labels clusters. On ne conserve que les documents qui sont dans catégorisés

In [16]:
clustersDocs: list[int] = labelsTousModeles['glove']['silhouette']
visualisation = preparation_LDAvis(clustersDocs)

C:\Users\atero\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
C:\Users\atero\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
C:\Users\atero\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [17]:
pyLDAvis.save_html(visualisation, "data/LDAvis/GloVe.html")
pyLDAvis.save_json(visualisation, "data/LDAvis/GloVe.json")
pyLDAvis.display(visualisation)